#Classificazione con rappresentazione SVD

Compito di classificazione del testo rispetto alle seguenti categorie:


*   Automotive
*   Patio, Lawn and Garden
*   Office Products
*   Toys_&_Games



Mount Drive

In [ ]:
import random
random.seed(2020)

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


Import librerie

In [ ]:
!pip install contractions

     |████████████████████████████████| 245kB 8.8MB/s 
     |████████████████████████████████| 317kB 12.0MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81701 sha256=30464c481eec06b8fca333aa075003c7e693cf912529c39ca297eab6c682d923
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
!pip install pyspellchecker

     |████████████████████████████████| 1.9MB 4.7MB/s 


In [ ]:
import gzip
import numpy as np
import json
import pandas as pd
from sklearn import utils
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords, treebank
import contractions
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()
from spellchecker import SpellChecker
from itertools import chain
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import seaborn as sns
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!pip install ipython-autotime
!pip install ipdb
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1831 sha256=e687478465be8b7ae5c494e0e33d2960d75a536b50af4729fdf58ab7ee55fbb4
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime
  Created wheel for ipdb: filename=ipdb-0.13.3-cp36-none-any.whl size=10847 sha256=d7ae1980eae6bf1e65373177142d5b2ed012b8eb654d7f7119586df928dd2ee7
  Stored in directory: /root/.cache/pip/wheels/75/00/30/4169bcc3643f0cf946dcf37af1b71364b390c4df91da02b03c
Successfully built ipdb


In [ ]:
pd.set_option('display.max_colwidth', -1)

time: 1.78 ms


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


##Caricamento dati

In [ ]:
collection_stem = pd.read_csv("/gdrive/My Drive/Progetto_Text_Mining/Collection/post_stopwords_stem_filtro.csv")
collection_lemma = pd.read_csv("/gdrive/My Drive/Progetto_Text_Mining/Collection/post_stopwords_lemma_filtro.csv")

time: 17.5 s


In [ ]:
print(len(collection_stem), len(collection_lemma))

111860 111860
time: 1.72 ms


In [ ]:
collection_stem.drop(columns=["Unnamed: 0"], inplace = True)
collection_lemma.drop(columns=["Unnamed: 0"], inplace = True)

time: 71.9 ms


In [ ]:
collection_lemma = collection_lemma.sample(frac = 1, random_state = 2020)
collection_stem = collection_stem.sample(frac = 1, random_state = 2020)

time: 137 ms


In [ ]:
def tostring(tokens):
  tokens = str(tokens)
  tokens = tokens.replace("[", "")
  tokens = tokens.replace("]", "")
  tokens = tokens.replace("'", "")
  tokens = tokens.replace(",", "")
  return tokens

time: 3.63 ms


##Funzione di CV

In [ ]:
!pip install XGBoost

time: 1.95 s


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score

def Cross_Validation(features, target, classificatore):
  kfold = StratifiedKFold(n_splits=5, random_state=2020, shuffle=True)
  cvscores = []

  i = 1
  for train, test in kfold.split(features, target):
    X_train, X_val = features[train], features[test]
    y_train, y_val = target.values[list(train)], target.values[list(test)]

    if str(type(classificatore)) == "<class 'keras.engine.sequential.Sequential'>":
      label_encoder = LabelEncoder()
      y_train = label_encoder.fit_transform(y_train)
      y_val = label_encoder.fit_transform(y_val)
      # fit model
      classificatore.fit(X_train, y_train)
      # predict
      pred = classificatore.predict_class(X_val)
    else:
      # fit model
      classificatore.fit(X_train, y_train)
      # predict
      pred = classificatore.predict(X_val)
    
    accuracy = accuracy_score(y_val, pred)
    print('Iterazione numero: ' + str(i))
    print('Accuracy attuale del modello di classificazione: ' + str(accuracy))

    cvscores.append(accuracy)
    i +=1

  print("Il punteggio medio ottenuto dalla cross-validation è " + str(np.mean(cvscores)) + " con una deviazione di " + str(np.std(cvscores)))
  return cvscores


time: 16.7 ms


Dataframe per risultati

In [ ]:
results = pd.DataFrame(columns=["Algoritmo", "Rappresentazione", "Punteggio_cv"])
results

,Algoritmo,Rappresentazione,Punteggio_cv


time: 24.9 ms


#Creazione matrice con SVD e Classificazione

##Lemma No-filter

In [ ]:
results = pd.DataFrame(columns=["Algoritmo", "Rappresentazione", "Punteggio_cv"])

time: 6.41 ms


In [ ]:
tfidf = TfidfVectorizer()
tfidfmatrix_lemma = tfidf.fit_transform(map(tostring, collection_lemma["lemmanosw"]))
tfidfmatrix_lemma.shape

(111860, 77440)

time: 8.31 s


In [ ]:
filetfidflemma = open("/gdrive/My Drive/Progetto_Text_Mining/Collection/tfidf_lemma.csv", "wb")
pickle.dump(tfidfmatrix_lemma, filetfidflemma)
filetfidflemma.close()

time: 3.54 s


In [ ]:
#imposto svd con numero features
svd = TruncatedSVD(100)
#creazione di una pipeline per normalizzare la matrice
lsa = make_pipeline(svd, Normalizer(copy=False))
lsa_lemma_nofiltro = lsa.fit_transform(tfidfmatrix_lemma)
explained_variance = svd.explained_variance_ratio_.sum()
print("  Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))

  Explained variance of the SVD step: 15%
time: 20.3 s


In [ ]:
features_lemma = np.asarray(lsa_lemma_nofiltro)
target_lemma = collection_lemma.target

time: 1.69 ms


In [ ]:
from sklearn.ensemble import RandomForestClassifier

#creazione del modello per CV
rf = RandomForestClassifier(n_estimators=100)

#richiamo funzione per CV
res = dict({"Algoritmo": "RandomForest", "Rappresentazione": "SVD", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, rf)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8478455211871982
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8530752726622564
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.851868406937243
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8478902199177544
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8506615412122296
Il punteggio medio ottenuto dalla cross-validation è 0.8502681923833363 con una deviazione di 0.0021032925799180256


,Algoritmo,Rappresentazione,Punteggio_cv
0,RandomForest,SVD,"[0.8478455211871982, 0.8530752726622564, 0.851868406937243, 0.8478902199177544, 0.8506615412122296]"


time: 13min 36s


In [ ]:
from xgboost import XGBClassifier

#creazione del modello per CV
xgbmodel = XGBClassifier(n_estimators=200)

#richiamo funzione per CV
res = dict({"Algoritmo": "XGBoost", "Rappresentazione": "SVD", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, xgbmodel)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8572322546039692
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8617468263901306
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8620150187734669
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8599141784373324
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8602270695512247
Il punteggio medio ottenuto dalla cross-validation è 0.8602270695512247 con una deviazione di 0.0017072340605889405


,Algoritmo,Rappresentazione,Punteggio_cv
0,XGBoost,SVD,"[0.8572322546039692, 0.8617468263901306, 0.8620150187734669, 0.8599141784373324, 0.8602270695512247]"


time: 49min 31s


In [ ]:
from sklearn import svm
SVM = svm.LinearSVC()
res = dict({"Algoritmo": "SVMLinear", "Rappresentazione": "SVD", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, SVM)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8679599499374218
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8727427141069194
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8709994636152333
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8683175397818702
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8683175397818702
Il punteggio medio ottenuto dalla cross-validation è 0.869667441444663 con una deviazione di 0.0018863499810923077


,Algoritmo,Rappresentazione,Punteggio_cv
0,SVMLinear,SVD,"[0.8679599499374218, 0.8727427141069194, 0.8709994636152333, 0.8683175397818702, 0.8683175397818702]"


time: 37.5 s


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 120)
res = dict({"Algoritmo": "KNN", "Rappresentazione": "SVD", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, KNN)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8266136241730735
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8274182013230824
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8301448238870016
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8237529054174861
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8263454317897372
Il punteggio medio ottenuto dalla cross-validation è 0.8268549973180761 con una deviazione di 0.002053341169283322


,Algoritmo,Rappresentazione,Punteggio_cv
0,KNN,SVD,"[0.8266136241730735, 0.8274182013230824, 0.8301448238870016, 0.8237529054174861, 0.8263454317897372]"


time: 57min 49s


In [ ]:
results.to_csv("/gdrive/My Drive/Progetto_Text_Mining/Collection/risultati_svd_lemma_nofiltro.csv")

time: 1.34 s


##Lemma Filter

In [ ]:
results = pd.DataFrame(columns=["Algoritmo", "Rappresentazione", "Punteggio_cv"])

time: 7.01 ms


In [ ]:
tfidf = TfidfVectorizer()
tfidfmatrix_lemma_filtrata = tfidf.fit_transform(map(tostring, collection_lemma["lemmanosw2"]))
tfidfmatrix_lemma_filtrata.shape

(111860, 34393)

time: 4.58 s


In [ ]:
filetfidflemma = open("/gdrive/My Drive/Progetto_Text_Mining/Collection/tfidf_lemma_filtro.csv", "wb")
pickle.dump(tfidfmatrix_lemma_filtrata, filetfidflemma)
filetfidflemma.close()

time: 1.05 s


In [ ]:
#imposto svd con numero features
svd = TruncatedSVD(100)
#creazione di una pipeline per normalizzare la matrice
lsa = make_pipeline(svd, Normalizer(copy=False))
lsa_lemma_filtro = lsa.fit_transform(tfidfmatrix_lemma_filtrata)
explained_variance = svd.explained_variance_ratio_.sum()
print("  Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))

  Explained variance of the SVD step: 13%
time: 11.3 s


In [ ]:
features_lemma = np.asarray(lsa_lemma_filtro)
target_lemma = collection_lemma.target

time: 1.89 ms


In [ ]:
from sklearn.ensemble import RandomForestClassifier

#creazione del modello per CV
rf = RandomForestClassifier(n_estimators=100)

#richiamo funzione per CV
res = dict({"Algoritmo": "RandomForest", "Rappresentazione": "SVD", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, rf)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8569193634900769
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8630430895762561
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8591989987484355
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8570534596817451
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8574110495261934
Il punteggio medio ottenuto dalla cross-validation è 0.8587251922045415 con una deviazione di 0.0023087402758436284


,Algoritmo,Rappresentazione,Punteggio_cv
0,RandomForest,SVD,"[0.8569193634900769, 0.8630430895762561, 0.8591989987484355, 0.8570534596817451, 0.8574110495261934]"


time: 15min 33s


In [ ]:
from xgboost import XGBClassifier

#creazione del modello per CV
xgbmodel = XGBClassifier(n_estimators=200)

#richiamo funzione per CV
res = dict({"Algoritmo": "XGBoost", "Rappresentazione": "SVD", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, xgbmodel)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8566511711067406
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8626854997318076
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8592883962095477
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8579474342928661
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8590649025567674
Il punteggio medio ottenuto dalla cross-validation è 0.8591274807795457 con una deviazione di 0.0020106679477520308


,Algoritmo,Rappresentazione,Punteggio_cv
0,XGBoost,SVD,"[0.8566511711067406, 0.8626854997318076, 0.8592883962095477, 0.8579474342928661, 0.8590649025567674]"


time: 44min 54s


In [ ]:
from sklearn import svm
SVM = svm.LinearSVC()
res = dict({"Algoritmo": "SVMLinear", "Rappresentazione": "SVD", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, SVM)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8571428571428571
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8608975505095655
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8585285177900948
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.857813338101198
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8589755050956552
Il punteggio medio ottenuto dalla cross-validation è 0.8586715537278741 con una deviazione di 0.001276037201376724


,Algoritmo,Rappresentazione,Punteggio_cv
0,SVMLinear,SVD,"[0.8571428571428571, 0.8608975505095655, 0.8585285177900948, 0.857813338101198, 0.8589755050956552]"


time: 48.3 s


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 120)
res = dict({"Algoritmo": "KNN", "Rappresentazione": "SVD", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, KNN)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8060075093867334
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8139191846951547
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8079742535311997
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8077060611478634
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8129805113534776
Il punteggio medio ottenuto dalla cross-validation è 0.8097175040228857 con una deviazione di 0.0031352901692741557


,Algoritmo,Rappresentazione,Punteggio_cv
0,KNN,SVD,"[0.8060075093867334, 0.8139191846951547, 0.8079742535311997, 0.8077060611478634, 0.8129805113534776]"


time: 48min 19s


In [ ]:
results.to_csv("/gdrive/My Drive/Progetto_Text_Mining/Collection/risultati_svd_lemma_filtro.csv")

##Stem No-filter

In [ ]:
results = pd.DataFrame(columns=["Algoritmo", "Rappresentazione", "Punteggio_cv"])

time: 7.1 ms


In [ ]:
tfidf = TfidfVectorizer()
tfidfmatrix_stem = tfidf.fit_transform(map(tostring, collection_stem["stemnosw"]))
tfidfmatrix_stem.shape

(111860, 60183)

time: 9.07 s


In [ ]:
filetfidfstem = open("/gdrive/My Drive/Progetto_Text_Mining/Collection/tfidf_stem.csv", "wb")
pickle.dump(tfidfmatrix_stem, filetfidfstem)
filetfidfstem.close()

time: 2.74 s


In [ ]:
#imposto svd con numero features
svd = TruncatedSVD(100)
#creazione di una pipeline per normalizzare la matrice
lsa = make_pipeline(svd, Normalizer(copy=False))
lsa_stem_nofiltro = lsa.fit_transform(tfidfmatrix_stem)
explained_variance = svd.explained_variance_ratio_.sum()
print("  Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))

  Explained variance of the SVD step: 16%
time: 21.4 s


In [ ]:
features_stem = np.asarray(lsa_stem_nofiltro)
target_stem = collection_stem.target

time: 1.93 ms


In [ ]:
from sklearn.ensemble import RandomForestClassifier

#creazione del modello per CV
rf = RandomForestClassifier(n_estimators=100)

#richiamo funzione per CV
res = dict({"Algoritmo": "RandomForest", "Rappresentazione": "SVD", "Punteggio_cv": Cross_Validation(features_stem, target_stem, rf)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8471750402288575
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8534775612372608
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.851019131056678
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8475326300733059
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8488288932594314
Il punteggio medio ottenuto dalla cross-validation è 0.8496066511711067 con una deviazione di 0.002357890872163477


,Algoritmo,Rappresentazione,Punteggio_cv
0,RandomForest,SVD,"[0.8471750402288575, 0.8534775612372608, 0.851019131056678, 0.8475326300733059, 0.8488288932594314]"


time: 14min 56s


In [ ]:
from xgboost import XGBClassifier

#creazione del modello per CV
xgbmodel = XGBClassifier(n_estimators=200)

#richiamo funzione per CV
res = dict({"Algoritmo": "XGBoost", "Rappresentazione": "SVD", "Punteggio_cv": Cross_Validation(features_stem, target_stem, xgbmodel)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8567405685678526
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8621044162345789
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8607187555873413
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8586626139817629
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.85960128732344
Il punteggio medio ottenuto dalla cross-validation è 0.8595655283389952 con una deviazione di 0.0018205941657990672


,Algoritmo,Rappresentazione,Punteggio_cv
0,XGBoost,SVD,"[0.8567405685678526, 0.8621044162345789, 0.8607187555873413, 0.8586626139817629, 0.85960128732344]"


time: 52min 47s


In [ ]:
from sklearn import svm
SVM = svm.LinearSVC()
res = dict({"Algoritmo": "SVMLinear", "Rappresentazione": "SVD", "Punteggio_cv": Cross_Validation(features_stem, target_stem, SVM)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8655015197568389
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.870686572501341
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8693456105846594
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8679152512068657
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8686304308957625
Il punteggio medio ottenuto dalla cross-validation è 0.8684158769890935 con una deviazione di 0.001721081072928157


,Algoritmo,Rappresentazione,Punteggio_cv
0,SVMLinear,SVD,"[0.8655015197568389, 0.870686572501341, 0.8693456105846594, 0.8679152512068657, 0.8686304308957625]"


time: 40.6 s


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 120)
res = dict({"Algoritmo": "KNN", "Rappresentazione": "SVD", "Punteggio_cv": Cross_Validation(features_stem, target_stem, KNN)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8274182013230824
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8282674772036475
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8320221705703558
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.826166636867513
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8289826568925442
Il punteggio medio ottenuto dalla cross-validation è 0.8285714285714286 con una deviazione di 0.0019637755337498643


,Algoritmo,Rappresentazione,Punteggio_cv
0,KNN,SVD,"[0.8274182013230824, 0.8282674772036475, 0.8320221705703558, 0.826166636867513, 0.8289826568925442]"


time: 1h 4min 15s


In [ ]:
results.to_csv("/gdrive/My Drive/Progetto_Text_Mining/Collection/risultati_svd_stem_nofiltro.csv")

time: 298 ms


##Stem Filter

In [ ]:
results = pd.DataFrame(columns=["Algoritmo", "Rappresentazione", "Punteggio_cv"])

time: 6.33 ms


In [ ]:
tfidf = TfidfVectorizer()
tfidfmatrix_stem_filtrata = tfidf.fit_transform(map(tostring, collection_stem["stemnosw2"]))
tfidfmatrix_stem_filtrata.shape

(111860, 27467)

time: 5.71 s


In [ ]:
filetfidfstem = open("/gdrive/My Drive/Progetto_Text_Mining/Collection/tfidf_stem_filtro.csv", "wb")
pickle.dump(tfidfmatrix_stem_filtrata, filetfidfstem)
filetfidfstem.close()

time: 200 ms


In [ ]:
#imposto svd con numero features
svd = TruncatedSVD(100)
#creazione di una pipeline per normalizzare la matrice
lsa = make_pipeline(svd, Normalizer(copy=False))
lsa_stem_filtro = lsa.fit_transform(tfidfmatrix_stem_filtrata)
explained_variance = svd.explained_variance_ratio_.sum()
print("  Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))

  Explained variance of the SVD step: 13%
time: 13.7 s


In [ ]:
features_stem = np.asarray(lsa_stem_filtro)
target_stem = collection_stem.target

time: 1.24 ms


In [ ]:
from sklearn.ensemble import RandomForestClassifier

#creazione del modello per CV
rf = RandomForestClassifier(n_estimators=100)

#richiamo funzione per CV
res = dict({"Algoritmo": "RandomForest", "Rappresentazione": "SVD", "Punteggio_cv": Cross_Validation(features_stem, target_stem, rf)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8591096012873234
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8622832111568032
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8582603254067585
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8574110495261934
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.85960128732344
Il punteggio medio ottenuto dalla cross-validation è 0.8593330949401036 con una deviazione di 0.0016534905687153646


,Algoritmo,Rappresentazione,Punteggio_cv
0,RandomForest,SVD,"[0.8591096012873234, 0.8622832111568032, 0.8582603254067585, 0.8574110495261934, 0.85960128732344]"


time: 14min 46s


In [ ]:
from xgboost import XGBClassifier

#creazione del modello per CV
xgbmodel = XGBClassifier(n_estimators=200)

#richiamo funzione per CV
res = dict({"Algoritmo": "XGBoost", "Rappresentazione": "SVD", "Punteggio_cv": Cross_Validation(features_stem, target_stem, xgbmodel)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8602717682817808
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8606740568567852
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8588414089039871
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8600035758984445
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8617915251206866
Il punteggio medio ottenuto dalla cross-validation è 0.8603164670123368 con una deviazione di 0.0009570122432378209


,Algoritmo,Rappresentazione,Punteggio_cv
0,XGBoost,SVD,"[0.8602717682817808, 0.8606740568567852, 0.8588414089039871, 0.8600035758984445, 0.8617915251206866]"


time: 52min 38s


In [ ]:
from sklearn import svm
SVM = svm.LinearSVC()
res = dict({"Algoritmo": "SVMLinear", "Rappresentazione": "SVD", "Punteggio_cv": Cross_Validation(features_stem, target_stem, SVM)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.859511889862328
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.862149114965135
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8582156266762024
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8590649025567674
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.8604058644734489
Il punteggio medio ottenuto dalla cross-validation è 0.8598694797067765 con una deviazione di 0.001341259875115788


,Algoritmo,Rappresentazione,Punteggio_cv
0,SVMLinear,SVD,"[0.859511889862328, 0.862149114965135, 0.8582156266762024, 0.8590649025567674, 0.8604058644734489]"


time: 47.4 s


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 120)
res = dict({"Algoritmo": "KNN", "Rappresentazione": "SVD", "Punteggio_cv": Cross_Validation(features_stem, target_stem, KNN)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.8110584659395673
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.8145896656534954
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.8122206329340247
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.8132934024673699
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.812488825317361
Il punteggio medio ottenuto dalla cross-validation è 0.8127301984623637 con una deviazione di 0.0011736630382461128


,Algoritmo,Rappresentazione,Punteggio_cv
0,KNN,SVD,"[0.8110584659395673, 0.8145896656534954, 0.8122206329340247, 0.8132934024673699, 0.812488825317361]"


time: 1h 1min 40s


In [ ]:
results.to_csv("/gdrive/My Drive/Progetto_Text_Mining/Collection/risultati_svd_stem_filtro.csv")

time: 157 ms
